In [0]:
%pip install pydicom==3.0.1

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = logging.getLogger("zipdcm")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(filename)s:%(lineno)s - %(levelname)s - %(message)s')
# Create a handler (e.g., StreamHandler for console output)
handler = logging.StreamHandler()
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

In [0]:
from pyspark.sql.datasource import DataSource, DataSourceReader
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType
from zip_dcm_ds import ZipDCMDataSource

def test_single(spark):
    # test a single zip (with a dcm and a license file)
    df = (
        spark.read
            .option('numPartitions','1')
            .format("zipdcm")
            .load("./resources/dcms/3.5.574.1.3.9030958.6.376.2860280475000825621.zip")
    )
    result = df.collect()
    assert len(result) == 1
    print(result)

def test_folder(spark):
    # test on a folder of zips
    df = (
        spark.read
          .option('numPartitions','2')
          .format("zipdcm")
          .load("./resources/dcms")
    )
    df.write.format('delta').mode('overwrite').saveAsTable("douglas_moore.pixels.zipdcm_test")

def test_scale(spark, numPartitions=128):
    df = (
        spark.read
          .option('numPartitions',f'{numPartitions}')
          .format("zipdcm")
          .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload")
    )
    df.write.format('delta').mode('overwrite').saveAsTable("douglas_moore.pixels.zipdcm_test")

In [0]:
# how many zips do we have?
df = (
    spark.read.format("binaryFile")
        .option("pathGlobFilter", "*.zip")
        .option("recursiveFileLookup", "true")
        .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload").drop('content')
    )
df.count()

1416

In [0]:
import time
start_time = time.time()
total_worker_cores = 32

# Add our custom Python DataSource for DICMO files storaged in a Zip Archive
spark.dataSource.register(ZipDCMDataSource)

#test_single(spark)
#test_folder(spark)
test_scale(spark, 128)

end_time = time.time()
operation_time = end_time - start_time
logger.info(f"Operation time: {operation_time} seconds and {operation_time * total_worker_cores} core-seconds")

Operation time: 77.83218097686768 seconds and 2490.6297912597656 core-seconds


In [0]:
import time
start_time = time.time()
total_worker_cores = 32

# Add our custom Python DataSource for DICMO files storaged in a Zip Archive
spark.dataSource.register(ZipDCMDataSource)

#test_single(spark)
#test_folder(spark)
test_scale(spark, 64)

end_time = time.time()
operation_time = end_time - start_time
logger.info(f"Operation time: {operation_time} seconds and {operation_time * total_worker_cores} core-seconds")

2025-05-23 15:41:11,570 - command-2588019255038221-1900083354:14 - INFO - Operation time: 101.74083495140076 seconds and 3255.706718444824 core-seconds
2025-05-23 15:41:11,570 - command-2588019255038221-1900083354:14 - INFO - Operation time: 101.74083495140076 seconds and 3255.706718444824 core-seconds


## Summary stats

In [0]:
%sql
select count(distinct zipfile) zipfiles, count(distinct zipfile, dcmfile) dcmfiles, count(distinct rowid), count(1) records
from douglas_moore.pixels.zipdcm_test

zipfiles,dcmfiles,count(DISTINCT rowid),records
1416,107158,6077,107158


In [0]:
%sql
select * 
from douglas_moore.pixels.zipdcm_test
limit 100;

rowid zipfile dcmfile meta 1 /Volumes/hls_radiology/tcia/downloads/tciaDownload/1.1.146.0.1.4587991.5.760.4003968910029450044.zip 1-01.dcm {'00080005': {'vr': 'CS', 'Value': ['ISO_IR 100']}, '00080008': {'vr': 'CS', 'Value': ['ORIGINAL', 'PRIMARY']}, '00080012': {'vr': 'DA', 'Value': ['20130606']}, '00080013': {'vr': 'TM', 'Value': ['100720']}, '00080014': {'vr': 'UI', 'Value': ['1.1.146.0.1.4587991.5.760.2945203270696602023']}, '00080016': {'vr': 'UI', 'Value': ['1.2.840.10008.5.1.4.1.1.128']}, '00080018': {'vr': 'UI', 'Value': ['1.1.146.0.1.4587991.5.760.5797490902696674792']}, '00080020': {'vr': 'DA', 'Value': ['20130606']}, '00080021': {'vr': 'DA', 'Value': ['20130606']}, '00080022': {'vr': 'DA', 'Value': ['20130606']}, '00080023': {'vr': 'DA', 'Value': ['20130606']}, '00080030': {'vr': 'TM', 'Value': ['100720']}, '00080031': {'vr': 'TM', 'Value': ['100747']}, '00080032': {'vr': 'TM', 'Value': ['100915']}, '00080033': {'vr': 'TM', 'Value': ['100720']}, '00080050': {'vr': 'SH', 'Value': ['285B7772']}, '00080054': {'vr': 'AE', 'Value': ['EHASCC_SCU']}, '00080060': {'vr': 'CS', 'Value': ['PT']}, '00080070': {'vr': 'LO', 'Value': ['GE MEDICAL SYSTEMS']}, '00080080': {'vr': 'LO', 'Value': ['Estrada, Hunt and Sparks Community Clinic']}, '00080090': {'vr': 'PN', 'Value': [{'Alphabetic': 'BERG^JULIAN'}]}, '00081030': {'vr': 'LO', 'Value': ['BREAST PRONE for 185-41-3645']}, '0008103E': {'vr': 'LO', 'Value': ['PET AC 3DWB for Omar Callahan']}, '00081070': {'vr': 'PN', 'Value': [{'Alphabetic': 'THOMAS^MORGAN'}]}, '00081090': {'vr': 'LO', 'Value': ['Discovery STE']}, '00100010': {'vr': 'PN', 'Value': [{'Alphabetic': 'CALLAHAN^OMAR'}]}, '00100020': {'vr': 'LO', 'Value': ['6207698118']}, '00100030': {'vr': 'DA', 'Value': ['19510421']}, '00100040': {'vr': 'CS', 'Value': ['U']}, '00101010': {'vr': 'AS', 'Value': ['062Y']}, '00101020': {'vr': 'DS', 'Value': [1.668]}, '00101030': {'vr': 'DS', 'Value': [78.54]}, '00101040': {'vr': 'LO', 'Value': ['1426 Russell Dr Covington, VA 24426']}, '00102160': {'vr': 'SH', 'Value': ['Caucasian']}, '00102180': {'vr': 'SH', 'Value': ['Land']}, '001021B0': {'vr': 'LT', 'Value': ['BREAST PRON : Omar Callahan : 799-924-7785x669']}, '00130010': {'vr': 'LO', 'Value': ['CTP']}, '00131010': {'vr': 'LO', 'Value': ['MIDI-B-Synthetic-Test']}, '00131013': {'vr': 'LO', 'Value': ['29059656']}, '00180015': {'vr': 'CS', 'Value': ['BREAST']}, '00180050': {'vr': 'DS', 'Value': [3.27]}, '00180071': {'vr': 'CS', 'Value': ['TIME']}, '00180073': {'vr': 'CS', 'Value': ['MANU']}, '00180074': {'vr': 'IS', 'Value': [0]}, '00180075': {'vr': 'IS', 'Value': [0]}, '00181020': {'vr': 'LO', 'Value': ['41.04']}, '00181030': {'vr': 'LO', 'Value': ['PRONE BREAST for 6207698118']}, '00181060': {'vr': 'DS', 'Value': [0.0]}, '00181063': {'vr': 'DS', 'Value': [0.0]}, '00181083': {'vr': 'IS', 'Value': [0]}, '00181084': {'vr': 'IS', 'Value': [0]}, '00181100': {'vr': 'DS', 'Value': [700.0]}, '00181120': {'vr': 'DS', 'Value': [0.0]}, '00181147': {'vr': 'CS', 'Value': ['CYLINDRICAL RING']}, '00181149': {'vr': 'IS', 'Value': [700, 153]}, '00181181': {'vr': 'CS', 'Value': ['NONE']}, '00181242': {'vr': 'IS', 'Value': [120000]}, '00185100': {'vr': 'CS', 'Value': ['HFP']}, '0020000D': {'vr': 'UI', 'Value': ['1.1.146.0.1.4587991.5.760.3172376819166170988']}, '0020000E': {'vr': 'UI', 'Value': ['1.1.146.0.1.4587991.5.760.4003968910029450044']}, '00200010': {'vr': 'SH'}, '00200011': {'vr': 'IS', 'Value': [20130606]}, '00200013': {'vr': 'IS', 'Value': [1]}, '00200032': {'vr': 'DS', 'Value': [347.265625, 347.265625, 46.0199966430664]}, '00200037': {'vr': 'DS', 'Value': [-1.0, 0.0, 0.0, 0.0, -1.0, 0.0]}, '00200052': {'vr': 'UI', 'Value': ['1.1.146.0.1.4587991.5.760.5005139650975466945']}, '00201040': {'vr': 'LO', 'Value': ['V']}, '00201041': {'vr': 'DS', 'Value': [46.02]}, '00280002': {'vr': 'US', 'Value': [1]}, '00280004': {'vr': 'CS', 'Value': ['MONOCHROME2']}, '00280010': {'vr': 'US', 'Value': [128]}, '00280011': {'vr': 'US', 'Value': [128]}, '00280030': {